## Datasets

**currentOpenInspections.csv**

Column information for this dataset can be found in [Data_Dictionary_-_Open_Restaurants_Inspections.xlsx](data/Data_Dictionary_-_Open_Restaurants_Inspections.xlsx). Note that the columns listed in the sheet matching until *ZIPCode* which is actually named *Postcode*. In addition there are 8 extra columns not listed: Latitude, Longitude, CommunityBoard, CouncilDistrict, CensusTract, BIN, BBL, NTA. The meanings of these could probably be extracted from the other .xlsx files.

**Open_Restaurant_Applications__Historic__20231202.csv**

Column information for this dataset can be found in [Open_Restaurant_Applications_Historical_Data_Dictionary.xlsx](data/Open_Restaurant_Applications_Historical_Data_Dictionary.xlsx)

**DOHMH_New_York_City_Restaraunt_Inspection_Results_20231202.csv**

Column information for this dataset can be found in [RestaurantInspectionDataDictionary_09242018.xlsx](data/RestaurantInspectionDataDictionary_09242018.xlsx). Note that there are columns in addition to the ones described in the sheet: Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1.

In [2]:
import jellyfish as jf
import pandas as pd

from enum import Enum


class ColumnType(Enum):
    CONTINUOUS = 1
    CATEGORICAL = 2
    OTHER = 3

In [3]:
df_cur = pd.read_csv('data/currentOpenInspections.csv')
df_doh = pd.read_csv('data/DOHMH_New_York_City_Restaurant_Inspection_Results_20231202.csv')
df_ope = pd.read_csv('data/Open_Restaurant_Applications__Historic__20231202.csv')

In [4]:
summary_data = [df_cur.shape, df_doh.shape, df_ope.shape]
df_summary = pd.DataFrame(summary_data, columns=['Rows', 'Columns'], index=[
                          'currentOpenInspections', 'DOHMH_New_York_City_Restaurant_Inspection_Results_20231202', 'Open_Restaurant_Applications__Historic__20231202'])
print(df_summary)

                                                      Rows  Columns
currentOpenInspections                               79203       20
DOHMH_New_York_City_Restaurant_Inspection_Resul...  209789       27
Open_Restaurant_Applications__Historic__20231202     14428       35


In [5]:
print(df_cur.columns)
print(df_doh.columns)
print(df_ope.columns)

Index(['Borough', 'RestaurantName', 'SeatingChoice', 'LegalBusinessName',
       'BusinessAddress', 'RestaurantInspectionID', 'IsSidewayCompliant',
       'IsRoadwayCompliant', 'SkippedReason', 'InspectedOn', 'AgencyCode',
       'Postcode', 'Latitude', 'Longitude', 'CommunityBoard',
       'CouncilDistrict', 'CensusTract', 'BIN', 'BBL', 'NTA'],
      dtype='object')
Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE', 'Latitude', 'Longitude',
       'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL',
       'NTA', 'Location Point1'],
      dtype='object')
Index(['objectid', 'globalid', 'Seating Interest (Sidewalk/Roadway/Both)',
       'Restaurant Name', 'Legal Business Name', 'Doing Business As (DBA)',
       'Building Number', 'Street', 'Borough', 'Po

In [6]:
def summarize_column(column: pd.Series, type: ColumnType, large_category: bool = False):
    '''
    This function returns information about the column based on the type
    '''

    if type == ColumnType.CATEGORICAL and not large_category:
        print('Total Count: ', len(column))
        print('Unique:', column.unique(), 'Count: ', len(column.unique()))
        print(column.value_counts())
        print('Type', column.dtype)
        print('Null count: ', column.isnull().sum())
        column.value_counts().plot(kind='pie')
    elif type == ColumnType.CATEGORICAL and large_category:
        print('Total Count: ', len(column))
        print('Unique Count: ', len(column.unique()))
        print('Type', column.dtype)
        print('Null count: ', column.isnull().sum())
    elif type == ColumnType.CONTINUOUS:
        print('Total Count: ', len(column))
        print('Null count: ', column.isnull().sum())
        print(column.describe())



In [7]:
summarize_column(df_cur['BBL'], ColumnType.CONTINUOUS, large_category=True)

Total Count:  79203
Null count:  8304
count    7.089900e+04
mean     2.167645e+09
std      1.240291e+09
min      0.000000e+00
25%      1.008370e+09
50%      2.023990e+09
75%      3.041460e+09
max      5.080430e+09
Name: BBL, dtype: float64


**Question** Is there a non-null Skipped Reason value only when *IsRoadwayCompliant* is equal to 'Skipped Intersection'?

Yes.

In [8]:
((df_cur['IsRoadwayCompliant'] == 'Skipped Inspection') == df_cur['SkippedReason'].notnull()).sum() == len(df_cur)

True

**Question**: Are there the same number of distinct *BusinessAddress*, *RestaurantName*, and *LegalBusinessName*?

No.

In [9]:
print('Unique Values')
print('-------------')
print('# Business Addresses:', len(df_cur['BusinessAddress'].unique()))
print('# Restaurant Names:', len(df_cur['RestaurantName'].unique()))
print('# Legal Business Names:', len(df_cur['LegalBusinessName'].unique()))

print('\nNull Values')
print('-----------')
print('# Business Addresses:', df_cur['BusinessAddress'].isnull().sum())
print('# Restaurant Names:', df_cur['RestaurantName'].isnull().sum())
print('# Legal Business Names:', df_cur['LegalBusinessName'].isnull().sum())

Unique Values
-------------
# Business Addresses: 10600
# Restaurant Names: 9839
# Legal Business Names: 10288

Null Values
-----------
# Business Addresses: 0
# Restaurant Names: 8
# Legal Business Names: 1


**Question**: Is latitude Null wherever longitude is NULL?

Yes.

In [10]:
(df_cur['Latitude'].isnull() == df_cur['Longitude'].isnull()).sum() == len(df_cur)

True

Notes:

* It would make sense if there were more restaurants than business addresses as there could be new restaraunts that start at the same location.
* The number of restaraunt names and legal business names don't match. There are more legal business names suggesting that there are matching errors

**Question**: Where Latitude, Longitude, CommunityBoard, CouncilDistrict, CensusTract, BIN, BBL, NTA all merged in a single `JOIN` to the rest of the data?

Yes.
 
* Latitude, Longitude, CommunityBoard, CouncilDistrict, CensusTract, and NTA have the same null values
* BIN and BBL have the same null values
* Where Latitude..NTA are NULL, are BIN and BBL also always NULL

In [11]:
if (df_cur['Latitude'].isnull() == df_cur['Longitude'].isnull()).sum() == len(df_cur) and \
(df_cur['Latitude'].isnull() == df_cur['CommunityBoard'].isnull()).sum() == len(df_cur) and \
(df_cur['Latitude'].isnull() == df_cur['CouncilDistrict'].isnull()).sum() == len(df_cur) and \
(df_cur['Latitude'].isnull() == df_cur['CensusTract'].isnull()).sum() == len(df_cur) and \
(df_cur['Latitude'].isnull() == df_cur['NTA'].isnull()).sum() == len(df_cur):
    print('Latitude, Longitude, CommunityBoard, CouncilDistrict, CensusTract, and NTA have the same null values')

if (df_cur['BIN'].isnull() == df_cur['BBL'].isnull()).sum() == len(df_cur):
    print('BIN and BBL have the same null values')

print('\nNull Values')
print('-----------')
print('# Latitude:', df_cur['Latitude'].isnull().sum())
print('# Longitude:', df_cur['Longitude'].isnull().sum())
print('# Community Board:', df_cur['CommunityBoard'].isnull().sum())
print('# Council District:', df_cur['CouncilDistrict'].isnull().sum())
print('# Census Tract:', df_cur['CensusTract'].isnull().sum())
print('# BIN:', df_cur['BIN'].isnull().sum())
print('# BBL:', df_cur['BBL'].isnull().sum())
print('# NTA:', df_cur['NTA'].isnull().sum())

Latitude, Longitude, CommunityBoard, CouncilDistrict, CensusTract, and NTA have the same null values
BIN and BBL have the same null values

Null Values
-----------
# Latitude: 7883
# Longitude: 7883
# Community Board: 7883
# Council District: 7883
# Census Tract: 7883
# BIN: 8304
# BBL: 8304
# NTA: 7883


In [12]:
print(len(df_cur[df_cur['Latitude'].isnull()].index.union(df_cur[df_cur['BIN'].isnull()].index)) == df_cur['BIN'].isnull().sum())
print(len(df_cur[df_cur['Latitude'].isnull()].index.union(df_cur[df_cur['BBL'].isnull()].index)) == df_cur['BBL'].isnull().sum())

True
True


**Question**: Is there anything interesting about the name (e.g. food truck) for those that do not have a CommunityBoard, NTA etc..? What about those that have a CommunityBoard, NTA, but no BIN or BBL?

Doesn't seem like it. See the image.


In [106]:
# Sample rows that have null values for Latitude
df_cur[df_cur['Latitude'].isnull()].sample(10)[['BusinessAddress', 'LegalBusinessName', 'RestaurantName']]

,BusinessAddress,LegalBusinessName,RestaurantName
21519,125 55th Street,Milos Inc.,Estiatorio Milos
19287,396 396 6th Avenue,Naybi restaurant corp,haab Mexican restaurant
68881,5420 54,Longford Inc,The Wild Goose
69714,161 west 161 west 231 st,New Adventure Restaurant,new adventure restaurant
25420,5 5 EAST 47TH STREET,NEW BLUE FLOWERS GOURMET CORP,CAFE ALICE
21942,5323 5323 8th Avenue,Empat Sekawan Corp,Langkawi Restaurants
71974,7901 New Utrecht,Father & sons pizzeria INC,Father & sons pizzeria INC
54290,194-21-23 NORTHERN BLVD,TWO DIMITRIS CORP,GYRO WORLD
18128,310 St Nicholas Ave Corner Bldg abutting Gates...,MAYA'S SNACK BAR BUSHWICK CORP,MAYA'S SNACK BAR BUSHWICK
15632,2083 Amsterdam w 163 street,LA CATRINA RESTAURANT MEXICAN DELI CORP,La Catrina Restaurant Deli


In [109]:
df_cur[df_cur['BIN'].isnull()].sample(10)[['BusinessAddress', 'LegalBusinessName', 'RestaurantName']]

,BusinessAddress,LegalBusinessName,RestaurantName
15533,101 127 street,Harlem ale house,Harlem Ale House
67318,175 175 Water Street,MANHATTAN BAKE LLC,Flavors
11640,206 206 Dyckman Street,Burgos Restaurant Corp,Burgos Restaurant
45453,2 Hope,DM Hope Alliance Inc,De mole Williamsburg
36159,206-210 118th Street,"Arts In Common, LLC",The Cecil Steakhouse
31594,41-06 41-06 greenpoint ave,"Peru Restaurant, Inc",Don Pollo
59367,431WEST 202 STREET S/W CORNER OF OUTSIDE PKING...,LILIANA VELAZQUEZ,LILIANA VELAZQUEZ
77523,521 Court,Cremini's Corp,Cremini's
57702,037040014 6908 Myrtle Avenue,5 time champs,Courtside lounge
67248,1b 344 West 38th St,"SUNFLOWER ESPRESSO BAR, CO",Sunflower Espresso Bar


![Don Pollo](assets/don_pollo.png)

**Question**: Could CensusTract be of type int?

In [13]:
(df_cur[df_cur['CensusTract'].notnull()]['CensusTract'].astype(int) == df_cur[df_cur['CensusTract'].notnull()]['CensusTract']).sum() == len(df_cur[df_cur['CensusTract'].notnull()])

True

**Question** Are the NTA neighborhoods truly unique, or are some misspelled?

Yes, it looks like each of the neighborhoods is truly unique and correct without any misspellings. The closest neighborhood is usually by swapping East with West or North with South.

In [22]:
# For each unique neighborhood in NTA, find the closest matching neighborhood using Levenshtein distance. Report those that have a distance less than 3.

for neighborhood in df_cur[df_cur['NTA'].notnull()]['NTA'].unique():
    for other_neighborhood in df_cur[df_cur['NTA'].notnull()]['NTA'].unique():
        if neighborhood != other_neighborhood and jf.levenshtein_distance(neighborhood, other_neighborhood) < 3:
            print('Neighborhood:', neighborhood, '| Other:', other_neighborhood)

Neighborhood: West Village | Other: East Village
Neighborhood: East Village | Other: West Village
Neighborhood: Bushwick North | Other: Bushwick South
Neighborhood: Bensonhurst West | Other: Bensonhurst East
Neighborhood: Bensonhurst East | Other: Bensonhurst West
Neighborhood: Sunset Park East | Other: Sunset Park West
Neighborhood: Crown Heights North | Other: Crown Heights South
Neighborhood: Washington Heights North | Other: Washington Heights South
Neighborhood: East Harlem South | Other: East Harlem North
Neighborhood: Bushwick South | Other: Bushwick North
Neighborhood: Sunset Park West | Other: Sunset Park East
Neighborhood: Washington Heights South | Other: Washington Heights North
Neighborhood: East Harlem North | Other: East Harlem South
Neighborhood: Crown Heights South | Other: Crown Heights North


**Question**: Can you show an example of a spelling error in restaraunt name?

In [34]:
# Group by Postcode and for each group, find a business that is mispelled with a Levenshtein distance of 1 or 2.

count = 0

for postcode in df_cur[df_cur['Postcode'].notnull()]['Postcode'].unique():
    for restaurant in df_cur[df_cur['Postcode'] == postcode]['RestaurantName'].unique():
        
        if pd.isnull(restaurant):
            continue
        
        for other_restaurant in df_cur[df_cur['Postcode'] == postcode]['RestaurantName'].unique():

            if pd.isnull(other_restaurant):
                continue

            if restaurant != other_restaurant and jf.levenshtein_distance(restaurant, other_restaurant) < 2:
                print('Restaurant:', restaurant, '| Other:', other_restaurant)
                count += 1

        if count >= 5:
            break

print('Total:', count)


Restaurant: Sam Won Garden | Other: Sam won Garden
Restaurant: Sam won Garden | Other: Sam Won Garden
Restaurant: Osteria 57 | Other: Osteria57
Restaurant: Osteria57 | Other: Osteria 57
Restaurant: PEPEROSSO TO GO | Other: PEPE ROSSO TO GO
Restaurant: pig n whistle | Other: Pig n whistle
Restaurant: Mama sushi | Other: Mamasushi
Restaurant: ROCATONE SEAFOOD RESTAURANT | Other: ROXATONE SEAFOOD RESTAURANT
Total: 8
